In [1]:
# import smtplib
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText
# from durable.lang import *
# import re

# # Email notification function
# def send_email_notification(subject, message, to_email):
#     try:
#         smtp_server = 'smtp.gmail.com'
#         smtp_port = 587
#         smtp_user = 'patilbk26@gmail.com'  # Your Gmail address
#         smtp_password = 'eiot gzcs krch cpbd'  # Use the app-specific password here

#         # Create the email
#         msg = MIMEMultipart()
#         msg['From'] = smtp_user
#         msg['To'] = to_email
#         msg['Subject'] = subject

#         # Add the email body
#         msg.attach(MIMEText(message, 'plain'))

#         # Connect to the server
#         server = smtplib.SMTP(smtp_server, smtp_port)
#         server.starttls()
#         server.login(smtp_user, smtp_password)

#         # Send the email
#         server.sendmail(smtp_user, to_email, msg.as_string())
#         server.quit()
#         print(f"Email sent to {to_email}")
#     except Exception as e:
#         print(f"Failed to send email: {e}")

# # Define the rule for project management
# with ruleset('project_management'):

#     @when_all((m.priority == 'High') & (m.type == 'Task') & (m.due_date.matches(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(\.\d{6})?')))
#     def handle_high_priority_task(c):
#         task_name = c.m.task_name
#         due_date = c.m.due_date
#         print(f"High priority task '{task_name}' is due on {due_date}.")

#         # Notify project manager (email instead of console output)
#         subject = f"Task '{task_name}' is Overdue"
#         message = f"Task '{task_name}' is overdue! Please take immediate action. Due date was {due_date}."
#         to_email = "basavaraj@v-align.in"
#         send_email_notification(subject, message, to_email)

#     @when_all((m.type == 'Project') & (m.status == 'Overdue'))
#     def handle_overdue_project(c):
#         project_name = c.m.project_name
#         print(f"Project '{project_name}' is overdue! Notifying project manager.")
#         subject = f"Project '{project_name}' is Overdue"
#         message = f"Project '{project_name}' is overdue! Please escalate."
#         to_email = "basavaraj@v-align.in"
#         send_email_notification(subject, message, to_email)

# # Example projects and tasks
# project1 = {'type': 'Project', 'project_name': 'Project Alpha', 'status': 'Overdue'}
# project2 = {'type': 'Project', 'project_name': 'Project Beta', 'status': 'On Time'}

# task1 = {'type': 'Task', 'task_name': 'Finalize Budget', 'priority': 'High', 'due_date': '2024-08-28T17:22:53.487205'}
# task2 = {'type': 'Task', 'task_name': 'Prepare Presentation', 'priority': 'Medium', 'due_date': '2024-09-01T12:00:00'}

# # Step 3: Post facts to the ruleset
# post('project_management', project1)
# post('project_management', task1)
# post('project_management', project2)


In [2]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from durable.lang import *
import psycopg2

# Database connection details
DATABASE_HOST = "database-test-postgress-instance.cpk2uyae6iza.ap-south-1.rds.amazonaws.com"
DATABASE_USERNAME = "postgres"
DATABASE_PASSWORD = "valign#123"
DATABASE_DB = "python_test_poc"
PORT = 5432

# Email notification function
def send_email_notification(subject, message, to_email):
    try:
        smtp_server = 'smtp.gmail.com'
        smtp_port = 587
        smtp_user = 'patilbk26@gmail.com'  # Your Gmail address
        smtp_password = 'eiot gzcs krch cpbd'  # Use the app-specific password here

        # Create the email
        msg = MIMEMultipart()
        msg['From'] = smtp_user
        msg['To'] = to_email
        msg['Subject'] = subject

        # Add the email body
        msg.attach(MIMEText(message, 'plain'))

        # Connect to the server
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(smtp_user, smtp_password)

        # Send the email
        server.sendmail(smtp_user, to_email, msg.as_string())
        server.quit()
        print(f"Email sent to {to_email}")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Function to connect to PostgreSQL database
def connect_to_db():
    try:
        conn = psycopg2.connect(
            dbname=DATABASE_DB,
            user=DATABASE_USERNAME,
            password=DATABASE_PASSWORD,
            host=DATABASE_HOST,
            port=PORT
        )
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to the database: {e}")
        raise

# Function to fetch real-time project data from the database
def fetch_projects_from_db():
    conn = connect_to_db()
    cursor = conn.cursor()

    # Assuming you have a projects table with project_name and status columns
    query = "SELECT project_name, status FROM projects"
    cursor.execute(query)
    projects = cursor.fetchall()

    project_list = []
    for project_name, status in projects:
        project_list.append({'type': 'Project', 'project_name': project_name, 'status': status})

    cursor.close()
    conn.close()

    return project_list

# Initialize email sending count
email_sent_count = 0

# Define the rule for project management
with ruleset('project_management'):
    
    # Handling Blocked projects
    @when_all((m.type == 'Project') & (m.status == 'Blocked'))
    def handle_blocked_project(c):
        global email_sent_count
        project_name = c.m.project_name
        print(f"Project '{project_name}' is Blocked!")

        # Send email only for the first 2 Blocked projects
        if email_sent_count < 2:
            subject = f"Project '{project_name}' is Blocked"
            message = f"Project '{project_name}' is Blocked! Please escalate."
            to_email = "basavaraj@v-align.in"
            send_email_notification(subject, message, to_email)
            email_sent_count += 1
        else:
            print(f"Project '{project_name}' Blocked, but no email sent as the limit of 2 emails has been reached.")

    # Pass all other project statuses
    @when_all((m.type == 'Project') & (m.status != 'Blocked'))
    def handle_other_project(c):
        print(f"Project '{c.m.project_name}' has status '{c.m.status}'. No action required.")

# Fetch real-time project data and post it to the ruleset
projects = fetch_projects_from_db()

for project in projects:
    post('project_management', project)


Project 'IIFL Samasta - Zoho Creator - Phase III Demand Tracker Enhancement' has status 'To Do'. No action required.
Project 'Zoho CRM - NR - IOF Query Module' has status 'To Do'. No action required.
Project 'NR - Creation of different module for 3D Drawing Request' has status 'To Do'. No action required.
Project 'CEEW - Zoho ONE - ERP Implementation' has status 'Development'. No action required.
Project 'Aajeeth Innovation LLP - Zoho CRM Implementation' has status 'To Do'. No action required.
Project 'ROAR Design-Zoho Books, Zoho Projects, Zoho People & Zoho Payroll' has status 'To Do'. No action required.
Project 'Airowire - Zoho Analytics' has status 'Development'. No action required.
Project 'Mitsubishi - Zoho Creator_Modernization Forms' has status 'Completed'. No action required.
Project 'Durian Industries - Zoho CRM & Creator (RFQ)' has status 'To Do'. No action required.
Project 'AER Worldwide - Zoho Books' has status 'Requirement Gathering'. No action required.
Project 'Lotusp